# FRBID prediction phase on new candidate files

    Authors : Zafiirah Hosenie
    Email : zafiirah.hosenie@gmail.com or zafiirah.hosenie@postgrad.manchester.ac.uk
    Affiliation : The University of Manchester, UK.
    License : MIT
    Status : Under Development
    Description : Python implementation for FRBID: Fast Radio Burst Intelligent Distinguisher.
    This code is tested in Python 3 version 3.6


In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import pandas as pd
from FRBID_code.model import compile_model,model_save 
import matplotlib.pylab as plt
from keras.utils import np_utils
from time import gmtime, strftime
from FRBID_code.util import makedirs, ensure_dir
from FRBID_code.prediction_phase import load_candidate, FRB_prediction


Using TensorFlow backend.


In [2]:
# Parameters to change
data_dir = './data/test_set/' # The directory where the hdf5 candidates are located
result_dir = './data/results_csv/' # The directory where the csv file after prediction will be saved
n_images = 'dm_fq_time' #can take str as either 'dm_fq_time' or 'dm_time' or 'fq_time'
model_cnn_name = 'NET3' # The network name choose from: 'NET1_32_64','NET1_64_128','NET1_128_256','NET2','NET3'
probability = 0.5 # The detection threshold

# Load the new candidates
- data_dir: The directory that contains the hdf5 files
- n_images: can either take str 'dm_fq_time', 'dm_time', 'fq_time'

In [3]:
test, ID_test = load_candidate(data_dir=data_dir ,n_images=n_images)
print("Total number of candidate instances: {}".format(str(len(ID_test))))
print("The Shape of the test set is {}".format(test.shape))

Total number of candidate instances: 2806
The Shape of the test set is (2806, 256, 256, 2)


# Prediction on new candidate files
Here we will load the pre-existing train model using the parameter 
INPUT:
- model_name: 'NET1_32_64', 'NET1_64_128', 'NET1_128_256', 'NET2', 'NET3'
- X_test : Image data should have shape (Nimages,256,256,2), (Nimages,256,256,1). This will vary depending on the criteria one use for n_images.
- ID: The candidate filename
- result_dir: The directory to save the csv prediction file

OUTPUT:
- overall_real_prob: An array of probability that each source is FRB. Value will range between [0 to 1.0]
- overall_dataframe: A table with column candidate name of all sources and its associated probability that it is a FRB source and its labels


In [4]:
overall_real_prob, overall_dataframe = FRB_prediction(model_name=model_cnn_name, X_test=test,ID=ID_test,result_dir=result_dir,probability=probability)


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Loaded model:NET3 from disk
2806/2806 [==============================] - 5s 2ms/step


In [5]:
# The transient ID for each candidate
ID_test

array(['0.015924_DM_221.96_beam_1C2020_02_02_11:55:53.fil.hdf5',
       '0.016748_DM_430.23_beam_0I2020_01_12_23:36:46.fil.hdf5',
       '0.018379_DM_148.90_beam_2C2020_02_14_04:34:59.fil.hdf5', ...,
       '0.994279_DM_271.08_beam_34C2019_12_18_08:41:48.fil.hdf5',
       '0.994432_DM_93.94_beam_0C2020_04_19_18:42:44.fil.hdf5',
       '0.999174_DM_36.23_beam_0I2020_04_13_01:04:46.fil.hdf5'],
      dtype='<U59')

In [6]:
# The probability that each source is a real source: It varies from 0 to 1
overall_real_prob

array([2.0953923e-06, 6.0713518e-04, 2.0273919e-04, ..., 3.6335757e-04,
       9.9999952e-01, 1.9407138e-02], dtype=float32)

In [7]:
# A dataframe that contains the transient ID and its probability that it is a Real source
overall_dataframe.iloc[1900:,:]

,candidate,probability,label
1900,0.500004_DM_238.23_beam_0C2020_04_11_21:37:49....,1.000000,1.0
1901,0.500004_DM_238.85_beam_0C2020_04_11_21:33:08....,1.000000,1.0
1902,0.500004_DM_241.30_beam_0C2020_04_12_04:29:27....,1.000000,1.0
1903,0.500004_DM_241.30_beam_0C2020_04_12_04:32:15....,0.999996,1.0
1904,0.500004_DM_241.30_beam_0C2020_04_12_04:54:33....,0.999998,1.0
...,...,...,...
2801,0.993664_DM_69.38_beam_7C2020_02_02_04:26:37.f...,0.000183,0.0
2802,0.993667_DM_2737.80_beam_195C2020_05_31_18:22:...,0.000014,0.0
2803,0.994279_DM_271.08_beam_34C2019_12_18_08:41:48...,0.000363,0.0
2804,0.994432_DM_93.94_beam_0C2020_04_19_18:42:44.f...,1.000000,1.0
